In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import svm
import sys
from os import remove, removedirs
from os.path import join, isfile, dirname
from time import time

In [5]:
#construct the ratings for the demo user
topMovies = """1,Toy Story (1995)
780,Independence Day (a.k.a. ID4) (1996)
590,Dances with Wolves (1990)
1210,Star Wars: Episode VI - Return of the Jedi (1983)
648,Mission: Impossible (1996)
344,Ace Ventura: Pet Detective (1994)
165,Die Hard: With a Vengeance (1995)
153,Batman Forever (1995)
597,Pretty Woman (1990)
1580,Men in Black (1997)
231,Dumb & Dumber (1994)"""
__file__=""
parentDir = dirname(dirname(__file__))
ratingsFile = join(parentDir, "personalRatings.txt")

if isfile(ratingsFile):
    r = raw_input("Looks like you've already rated the movies. Overwrite ratings (y/N)? ")
    if r and r[0].lower() == "y":
        remove(ratingsFile)
    else:
        sys.exit()

prompt = "Please rate the following movie (1-5 (best), or 0 if not seen): "
print prompt

now = int(time())
n = 0

f = open(ratingsFile, 'w')
for line in topMovies.split("\n"):
    ls = line.strip().split(",")
    valid = False
    while not valid:
        rStr = raw_input(ls[1] + ": ")
        r = int(rStr) if rStr.isdigit() else -1
        if r < 0 or r > 5:
            print prompt
        else:
            valid = True
            if r > 0:
                f.write("0::%s::%d::%d\n" % (ls[0], r, now))
                n += 1
f.close()

if n == 0:
    print "No rating provided!"

Looks like you've already rated the movies. Overwrite ratings (y/N)? 


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [2]:
mnames=['user_id','movie_id','movie_rating','timestamp']
demo_rate=pd.read_table('personalRatings.txt',sep='::', engine='python',header=None, names=mnames)

In [3]:
mnames=['user_id','movie_id','movie_rating','timestamp']
movie_rate=pd.read_table('hetrec/user_ratedmovies-timestamps.dat',sep='\t', engine='python',header=None, names=mnames)

In [4]:
mnames=['movie_id','tag_id','tag_weight']
movie_tags=pd.read_table('hetrec/movie_tags.dat',sep='\t', engine='python',header=None, names=mnames)

In [5]:
mnames=['movie_id','country']
movie_country=pd.read_table('hetrec/movie_countries.dat',sep='\t', engine='python',header=None, names=mnames)

In [6]:
mnames=['movie_id','director_id','directorName']
movie_director=pd.read_table('hetrec/movie_directors.dat',sep='\t', engine='python',header=None, names=mnames)

In [7]:
mnames=['movie_id','genre']
movie_genre=pd.read_table('hetrec/movie_genres.dat',sep='\t', engine='python',header=None, names=mnames)

In [8]:
movieActor = {}
actorDict = {};
def readActors():
    fobj = open('hetrec/movie_actors.dat', 'r');
    fobj.readline();
    while True:
        s = fobj.readline();
        if len(s) == 0:
            break;
        li = s.split('\t');
        movID = li[0]; actorID = li[1]; ranking = li[3];
        movID = int(movID); ranking = int(ranking);
        if movID in movieActor:
            movieActor[movID].append((actorID, ranking));
        else:
            movieActor[movID] = [(actorID, ranking)];
        
        actorDict[actorID] = 0;
            
    fobj.close();
    
    
readActors()

In [9]:
movie_actors = pd.DataFrame([[col1,col2,col3] for col1, d in movieActor.items() for col2, col3 in d]
                            ,columns=['movie_id','actor','actor_rating'])
del actorDict
del col1, col2,col3
del d
del movieActor
#movie_actors.head()


In [10]:
movie_country=movie_country[movie_country.index!=0]
movie_director=movie_director[movie_director.index!=0]
movie_genre=movie_genre[movie_genre.index!=0]
movie_rate=movie_rate[movie_rate.index!=0]
movie_tags=movie_tags[movie_tags.index!=0]
del movie_director['directorName']

In [11]:
tags_pivot=movie_tags.pivot(index='movie_id', columns='tag_id', values='tag_weight')
tags_pivot=tags_pivot.fillna(0)

In [12]:
rates_pivot=movie_rate.pivot(index='movie_id', columns='user_id', values='movie_rating')
rates_pivot=rates_pivot.fillna(0)

In [13]:
nrows, ncols=rates_pivot.shape

In [ ]:
#ridge regression
#randomly get a users ratings
sample_size=20
random_user=np.random.randint(0,high=ncols,size=sample_size)
pred=np.zeros(1,dtype=float)
actual=np.zeros(1,dtype=float)
#here put the loop
for i in range(1,sample_size+1):
    #print i
    user_rating=rates_pivot.ix[:,random_user[0]]
    user_rating=user_rating.fillna(0)
    user_rating=user_rating[user_rating>0]
    #print type(user_rating)
    #randomly split the user data into test and train
    random_movies=random.sample(user_rating.index, user_rating.size/5)
    test_target=user_rating.ix[random_movies].astype(float)
    #test=random_sample.ix[random.sample(random_sample.index, 10)]
    train_target=user_rating.drop(random_movies).astype(float)
    test_features=tags_pivot.ix[test_target.index]
    test_features=test_features.fillna(0)
    test_features=test_features.values.astype(float)
    train_features=tags_pivot.ix[train_target.index]
    train_features=train_features.fillna(0)
    train_features=train_features.values.astype(float)
    clf = linear_model.Ridge (alpha=100)
    clf.fit (train_features,train_target.values.astype(float))
    #clf.predict(test_features)
    #print mean_squared_error(test_target.values.astype(float), clf.predict(test_features))
    actual=np.append(actual,test_target.values.astype(float))
    pred=np.append(pred,clf.predict(test_features))

In [ ]:
mean_squared_error(actual,pred)

In [ ]:
#Lasso regression
#randomly get a users ratings
sample_size=20
random_user=np.random.randint(0,high=ncols,size=sample_size)
pred=np.zeros(1,dtype=float)
actual=np.zeros(1,dtype=float)
#here put the loop
for i in range(1,sample_size+1):
    #print i
    user_rating=rates_pivot.ix[:,random_user[0]]
    user_rating=user_rating.fillna(0)
    user_rating=user_rating[user_rating>0]
    #print type(user_rating)
    #randomly split the user data into test and train
    random_movies=random.sample(user_rating.index, user_rating.size/5)
    test_target=user_rating.ix[random_movies].astype(float)
    #test=random_sample.ix[random.sample(random_sample.index, 10)]
    train_target=user_rating.drop(random_movies).astype(float)
    test_features=tags_pivot.ix[test_target.index]
    test_features=test_features.fillna(0)
    test_features=test_features.values.astype(float)
    train_features=tags_pivot.ix[train_target.index]
    train_features=train_features.fillna(0)
    train_features=train_features.values.astype(float)
    clf = linear_model.Lasso(alpha = 0.1)
    clf.fit (train_features,train_target.values.astype(float))
    #clf.predict(test_features)
    #print mean_squared_error(test_target.values.astype(float), clf.predict(test_features))
    actual=np.append(actual,test_target.values.astype(float))
    pred=np.append(pred,clf.predict(test_features))

In [ ]:
mean_squared_error(actual,pred)

In [17]:
#SV regression
#randomly get a users ratings
sample_size=2
random_user=np.random.randint(0,high=ncols,size=sample_size)
pred=np.zeros(1,dtype=float)
actual=np.zeros(1,dtype=float)
#here put the loop
for i in range(1,sample_size+1):
    #print i
    user_rating=rates_pivot.ix[:,random_user[0]]
    user_rating=user_rating.fillna(0)
    user_rating=user_rating[user_rating>0]
    #print type(user_rating)
    #randomly split the user data into test and train
    random_movies=random.sample(user_rating.index, user_rating.size/5)
    test_target=user_rating.ix[random_movies].astype(float)
    #test=random_sample.ix[random.sample(random_sample.index, 10)]
    train_target=user_rating.drop(random_movies).astype(float)
    test_features=tags_pivot.ix[test_target.index]
    test_features=test_features.fillna(0)
    test_features=test_features.values.astype(float)
    train_features=tags_pivot.ix[train_target.index]
    train_features=train_features.fillna(0)
    train_features=train_features.values.astype(float)
    clf = svm.SVR()
    clf.fit (train_features,train_target.values.astype(float))
    #clf.predict(test_features)
    #print mean_squared_error(test_target.values.astype(float), clf.predict(test_features))
    actual=np.append(actual,test_target.values.astype(float))
    pred=np.append(pred,clf.predict(test_features))

In [18]:
mean_squared_error(actual,pred)

1.2511432595808303